### Benchmariking Spread trading  ----- Narendra Kunapareddy, 455474

In [1]:
%matplotlib inline

In [281]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import Quandl
import functools
import seaborn as sns
import ggplot as gg
import datetime as dt
import calendar as cal
import statsmodels.formula.api as smf

In [72]:
@functools.lru_cache(maxsize = 16)
def fetch_quandl(my_securities, start_date, end_date):
    qdata = Quandl.get(list(my_securities), returns="pandas", trim_start=start_date, trim_end=end_date,authtoken = "sw1wiPPJnCGXBnPdZG2k")
    return qdata

In [73]:
def clean_quandl_data(dataframe):
    replacement_columns = {}
    for c in dataframe.columns:
        series_name, variable = c.split(' - ')
        source_name, asset = series_name.split('.')
        replacement_columns[c] = asset+":"+variable
    renamed_data = dataframe.rename(columns=replacement_columns)
    return renamed_data

In [308]:
etf_pairs = (("RYU","XLU"),("IST","IYZ"),("RING","GDX"),("XSD","SMH"),("PBE","XBI"),("IEO","XOP"),("PXJ","OIH"),
             ("RTH","XRT"),("SIVR","SLV"),("HYLD","JNK"))
# Get Fama french factors returns
ff_returns = Quandl.get('KFRENCH/FACTORS_D', returns="pandas", trim_start="2013-12-02", trim_end="2015-12-31",
                        authtoken = "sw1wiPPJnCGXBnPdZG2k")
# annualize ff_returns and convert from percentage to returns
ff_returns = ff_returns * 252/100
ff_returns = ff_returns.rename(columns = {'Mkt-RF':'MktRf'})
M = 20
s = np.inf
u_lim = [0.01,0.02,0.01,0.02,0.025,0.025,0.005,0.02,0.002,0.005]
l_lim = [-0.005,-0.02,-0.005,-0.01,-0.025,-0.005,-0.015,-0.01,-0.002,-0.005]

In [309]:
def spread_analysis(spread,M,g,j,s,K,showplot):
    # Dataframe containing all the trades
    trades = pd.DataFrame(columns=["PositionType","Entry_Date","Entry_Spread","Nx","Ny","Exit_Date","Exit_Spread","PNL",
                                  "CapitalUntilExit"])
    # returns on most of the days is zero
    daily_returns = pd.DataFrame({"Returns":[0.0]* len(spread)}, index= spread.index)
    trade_num = 0
    # Keep track if position is open or closed
    open = False
    
    for currentDate,val in spread.iteritems():
        
        if not(open) and val > g and not(lastDayOfMonth[spread.index.get_loc(currentDate)]):
            # Open a new position if no position and spread > g and if current date is not last day of month
            Qx = Nx[currentDate]
            Qy = Ny[currentDate]
            trades.loc[trade_num] = ["Short",currentDate,val,Qx,Qy,np.nan, np.nan,np.nan, np.nan]
            xVal = Qx* price_data[etf1+":Adj_Close"][currentDate]
            yVal = Qy* price_data[etf2+":Adj_Close"][currentDate]
            entryVal = xVal - yVal  # Should be very less
            grossCash = xVal + yVal
            open = True
          
        elif open:
            # Check for stop loss barrier
            currentVal = -Qx* price_data[etf1+":Adj_Close"][currentDate] + Qy* price_data[etf2+":Adj_Close"][currentDate]
            aboveStopLoss = (-(currentVal+entryVal) * 100/grossCash) > s
                
            # Close the positon if it is end of month or less than j or exceeded stop loss barrier
            if lastDayOfMonth[spread.index.get_loc(currentDate)] or val < j or aboveStopLoss:
                trades.set_value(trade_num, "Exit_Date",currentDate)
                trades.set_value(trade_num, "Exit_Spread",val)

                # Calculate PNL & close the position
                PNL = currentVal + entryVal
                trades.set_value(trade_num,"PNL",PNL)
                trades.set_value(trade_num,"CapitalUntilExit",K)
                # update to non zero returns on exit day and annualize
                daily_returns.set_value(currentDate,"Returns",PNL * 252/K)
                # update running capital
                K = K + PNL
                open = False
                trade_num += 1   
            
    if showplot:
        # Plot spread, entry and exit points
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1,1,1)
        p1 = spread.plot(label='Spread')
        p2 = plt.axhline(g,color='b')
        p3 = plt.axhline(j,color='r')
        p4, = ax.plot(trades["Entry_Date"],trades["Entry_Spread"],'b*',ms=15)
        p5, = ax.plot(trades["Exit_Date"],trades["Exit_Spread"],'r*',ms=15)
        plt.legend((p2,p3,p4,p5),('g','j','Entry','Exit'))
        plt.ylabel("Spread")
          
    return {"spread":spread,"trades":trades,"daily_returns":daily_returns}

In [313]:
# run through each spread returns and calculate spread returns
ratios = pd.DataFrame(columns=["SR","SO","Mkt_RF_SR","Mkt_RF_SO","SMB_SR","SMB_SO","HML_SR","HML_SO","Multi_SR","Multi_SO"])
for i in range(len(etf_pairs)):
    etf1 = etf_pairs[i][0]
    etf2 = etf_pairs[i][1]
    # get data of each pair
    price_data = fetch_quandl(("EOD/"+etf1,"EOD/"+etf2),"2013-12-02","2015-12-31")
    # clean price data of each pair
    price_data = clean_quandl_data(price_data)
    # calculate dollar volume of x of each pair
    price_data[etf1+":Dollar_Volume"]=price_data[etf1+":Adj_Close"] * price_data[etf1+":Adj_Volume"]
    # calculate 15 window running median of dollar volume
    Nt = price_data[etf1+":Dollar_Volume"].rolling(window=15).median()
    # set maximum capital
    K = np.max(Nt) *2
    # calculate time varying quantities of individual etfs of each pair
    Nx = ((Nt/100)/price_data[etf1+":Adj_Close"]).fillna(value = 0).astype(int)
    Ny = ((Nt/100)/price_data[etf2+":Adj_Close"]).fillna(value = 0).astype(int)
    
    # calculate daily log returns of X and Y of a pair
    logRetsX = np.log(price_data[etf1+":Adj_Close"]/price_data[etf1+":Adj_Close"].shift(1))
    logRetsY = np.log(price_data[etf2+":Adj_Close"]/price_data[etf2+":Adj_Close"].shift(1))
    
    # calculate spread returns 
    spread = logRetsX.rolling(window = M).sum()-logRetsY.rolling(window = M).sum()
    spread = spread[spread.index >= '2014-01-01']
    # list to determine if a day is end of that month
    lastDayOfMonth = spread.index[0:-1].month != spread.index[1:].month
    lastDayOfMonth = np.append(lastDayOfMonth, True)
    g =  u_lim[i]
    j =  l_lim[i]
    result = spread_analysis(spread, M, g, j, s, K,False)

    # collect all the returns to single dataframe
    returns = result["daily_returns"]["Returns"]
    if i == 0:
        etf_returns = pd.DataFrame({"-".join(etf_pairs[i]): returns})
    else:
        etf_returns["-".join(etf_pairs[i])] = returns
    
    
    ratios.loc[i] = [np.nan] * 10
    # calculate sharpe ratio
    ratios.set_value(i,"SR",np.mean(returns)/np.std(returns))
    # calculate sortino ratio
    ratios.set_value(i,"SO",np.mean(returns)/np.std(returns[returns<0]))
    
    # regression on Mkt-RF and calculate ratios
    mod = smf.ols(formula='returns~MktRf+0', data=pd.DataFrame({"returns":returns,"MktRf":ff_returns["MktRf"]}))
    residuals = mod.fit().resid
    ratios.set_value(i,"Mkt_RF_SR",np.mean(residuals)/np.std(residuals))
    ratios.set_value(i,"Mkt_RF_SO",np.mean(residuals)/np.std(residuals[residuals<0]))
    
    # regression on SMB and calculate ratios
    mod = smf.ols(formula='returns~SMB+0', data=pd.DataFrame({"returns":returns,"SMB":ff_returns["SMB"]}))
    residuals = mod.fit().resid
    ratios.set_value(i,"SMB_SR",np.mean(residuals)/np.std(residuals))
    ratios.set_value(i,"SMB_SO",np.mean(residuals)/np.std(residuals[residuals<0]))
    
    # regression on HML and calculate ratios
    mod = smf.ols(formula='returns~HML+0', data=pd.DataFrame({"returns":returns,"HML":ff_returns["HML"]}))
    residuals = mod.fit().resid
    ratios.set_value(i,"HML_SR",np.mean(residuals)/np.std(residuals))
    ratios.set_value(i,"HML_SO",np.mean(residuals)/np.std(residuals[residuals<0]))
    
    # Multi variate regression and calculate ratios
    mod = smf.ols(formula='returns~MktRf+SMB+HML+0', data=pd.DataFrame({"returns":returns,"MktRf":ff_returns["MktRf"],
                                                                       "SMB":ff_returns["SMB"],"HML":ff_returns["HML"]}))
    residuals = mod.fit().resid
    ratios.set_value(i,"Multi_SR",np.mean(residuals)/np.std(residuals))
    ratios.set_value(i,"Multi_SO",np.mean(residuals)/np.std(residuals[residuals<0]))

In [314]:
ratios

,SR,SO,Mkt_RF_SR,Mkt_RF_SO,SMB_SR,SMB_SO,HML_SR,HML_SO,Multi_SR,Multi_SO
0,$0.0268443969,$0.0082740079,$0.0243761941,$0.0381422912,$0.0238165414,$0.0354702510,$0.0266561909,$0.0418102137,$0.0195411183,$0.0303073717
1,$0.0359315544,$0.0189125531,$0.0347230398,$0.0526404231,$0.0390971201,$0.0564268913,$0.0360350638,$0.0486178951,$0.0403875335,$0.0574087006
2,$0.0848624528,$0.0291976808,$0.0855212440,$0.1513506686,$0.0846086134,$0.1580800134,$0.0817042971,$0.1634571334,$0.0808776611,$0.1598180380
3,$0.0152008471,$0.0052860963,$0.0135218131,$0.0153839762,$0.0121345976,$0.0133619562,$0.0162061581,$0.0162267744,$0.0100045401,$0.0113219810
4,$-0.0333043200,$-0.0067387900,$-0.0325288428,$-0.0245628539,$-0.0364382988,$-0.0292065595,$-0.0326805086,$-0.0241064285,$-0.0370908488,$-0.0303112383
5,$0.0013729013,$0.0004002888,$0.0002939667,$0.0003163234,$0.0008277452,$0.0008672424,$0.0026602077,$0.0025681852,$0.0012754652,$0.0013493537
6,$0.0844330707,$0.0445825048,$0.0839862595,$0.2379673527,$0.0842996010,$0.2321044741,$0.0818930824,$0.2385552688,$0.0805620315,$0.2375264421
7,$-0.0516030443,$-0.0091689029,$-0.0532293548,$-0.0415116115,$-0.0504745898,$-0.0367464891,$-0.0536026603,$-0.0423778862,$-0.0535354291,$-0.0417842837
8,$0.1544048747,NaN,$0.1570855818,$2.9140426286,$0.1552483930,$12.4181461601,$0.1536427406,$22.5582063750,$0.1570472549,$2.6498014009
9,$0.0673484254,$0.0268012260,$0.0659684120,$0.1166545895,$0.0698695466,$0.1165248723,$0.0668134770,$0.1208673909,$0.0697395829,$0.1187080211


### Above table contains results of spread analysis using different parameters. Number of trades and profits are sensitive to M,g,j,s parameters. 
### Very low value of stop loss is increasing number of trades and profits,  but profits can be nullified by transaction costs in reality. Stop loss impact is varying with the limits of g an j. If g is at high value and j is at low value then high stop loss percentage is resulting in negative profits as the number of trades are reducing and exiting at unfavorable points. 
### Change in M day returns also have impact on profit levels. At some levels of g and j, high value of M is resulting in negative PnL
### Spread under consideration is more on upside than on downside. So g and j levels are chosen accordingly to enter and exit the trades. If low j values are considered, then number of trades are zero in few cases
### Overall all the parameters have considerable impact on the final PnL